# Train First Model: Logistic Regression 

In [3]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=LSw7hQSUITaGjuhyYZM8b6X2GeogaypgYRraQW4JTqQ&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [reddit-master].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [5]:
!gcloud config set project reddit-master

Updated property [core/project].


In [4]:
import pandas as pd
import numpy as np
import pickle as pkl
import nltk.data
import joblib
import nltk
import ast
import logging


from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import datetime
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giuliagalli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def loggg(msg):
    print("[INFO] {}: {}".format(datetime.datetime.now(), msg))

As the df is too big to work the GridSearch with all data, I'd like to try it just for 2 subreddits

In [7]:
subreddits = ['aww', 'nba']#, 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [8]:
!ls

07.1_modelling_LRC_TFIDF.ipynb
07.2_modelling_SVM_TFIDF.ipynb
07.3_modelling_RandomForestClassifier_TFIDF.ipynb
Untitled.ipynb
comments_posts_Fitness.pkl
comments_posts_IAmA.pkl
comments_posts_atheism.pkl
comments_posts_aww.pkl
comments_posts_europe.pkl
comments_posts_funny.pkl
comments_posts_gaming.pkl
comments_posts_movies.pkl
comments_posts_nba.pkl
comments_posts_politics.pkl
comments_posts_science.pkl
comments_posts_technology.pkl
comments_posts_todayilearned.pkl
comments_posts_worldnews.pkl
model_svm.joblib
summary.txt


In [9]:
%%time

model_LR_df = pd.DataFrame(columns = ["subreddit", "body", "subreddit_id"])

for subreddit in subreddits:
    #!gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    loggg("comments_posts_" + subreddit + ".pkl downloaded")

    df = pd.read_pickle("comments_posts_" + subreddit + ".pkl")

    model_LR_df = pd.concat([model_LR_df, df], ignore_index=True)

[INFO] 2019-11-27 09:37:09.397511: comments_posts_aww.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


[INFO] 2019-11-27 09:37:12.483193: comments_posts_nba.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


CPU times: user 5.61 s, sys: 874 ms, total: 6.49 s
Wall time: 6.63 s


In [11]:
loggg(model_LR_df.shape)

[INFO] 2019-11-27 10:00:31.499115: (1547237, 4)


In [12]:
X = model_LR_df['body']
y = model_LR_df['subreddit_id']

# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)
    return dict(X_train=X_train,
                X_val=X_val,
                X_test=X_test,
                y_train=y_train.astype('int'),
                y_val=y_val.astype('int'),
                y_test=y_test.astype('int'))

In [13]:
parameters = {
"tfidf__max_df": [0.25, 0.5, 0.75],
"tfidf__ngram_range": [(1, 1), (1, 2)],
"clf__estimator__class_weight": ["balanced",None]
}



In [14]:
model_LR_df_split = train_dev_test_split(X,y)

In [15]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(preprocessor=" ".join, ngram_range=(1,2))),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='sag', class_weight="balanced", multi_class='auto')))
    ])


grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1, verbose=10)

In [16]:
loggg("pipeline created")

grid_search_tune.fit(model_LR_df_split['X_train'], model_LR_df_split['y_train'].to_numpy().astype('int'))

[INFO] 2019-11-27 10:01:28.141049: pipeline created
Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed: 13.5min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 15.6min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<built-in meth...ate=None,
          solver='sag', tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'tfidf__max_df': [0.25, 0.5, 0.75], 'tfidf__ngram_range': [(1, 1), (1, 2)], 'clf__estimator__class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [17]:
# measuring performance on test set
loggg('Best parameters set:')
print(grid_search_tune.best_estimator_.steps)
# measuring performance on test set
loggg('Applying best classifier on test data:')

[INFO] 2019-11-27 10:23:55.262423: Best parameters set:
[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.25, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<built-in method join of str object at 0x109c58e30>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)), ('clf', OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='auto', n_jobs=None, penalty='l2', random_state=None,
          solver='sag', tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))]
[INFO] 2019-11-27 10:23:55.263601: Applying best classifier on test data:


In [18]:
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(model_LR_df_split['X_test'])
print(classification_report(model_LR_df_split['y_test'], predictions))

              precision    recall  f1-score   support

           3       0.87      0.90      0.89    116073
           8       0.90      0.87      0.88    116013

   micro avg       0.89      0.89      0.89    232086
   macro avg       0.89      0.89      0.89    232086
weighted avg       0.89      0.89      0.89    232086



# All data

Now we try to train all the DataFrame

In [19]:
subreddits = ['aww', 'nba', 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [21]:
%%time

model_LR_df = pd.DataFrame(columns = ["subreddit", "body", "subreddit_id"])

for subreddit in subreddits:
    #!gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    loggg("comments_posts_" + subreddit + ".pkl downloaded")

    df = pd.read_pickle("comments_posts_" + subreddit + ".pkl")

    model_LR_df = pd.concat([model_LR_df, df], ignore_index=True)

[INFO] 2019-11-27 10:27:37.118482: comments_posts_aww.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


[INFO] 2019-11-27 10:27:39.817402: comments_posts_nba.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


[INFO] 2019-11-27 10:27:43.227748: comments_posts_movies.pkl downloaded


...

[INFO] 2019-11-27 10:31:41.434750: comments_posts_funny.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


[INFO] 2019-11-27 10:31:50.569051: comments_posts_gaming.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


CPU times: user 1min 48s, sys: 2min 5s, total: 3min 54s
Wall time: 4min 32s


In [24]:
loggg(model_LR_df.shape)

[INFO] 2019-11-27 10:32:18.892564: (10833350, 4)


In [27]:
X = model_LR_df['body']
y = model_LR_df['subreddit_id']

# Definint a fucntion that slit the dataset into three subsets: train and test
def train_dev_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    return dict(X_train=X_train,
                X_test=X_test,
                y_train=y_train.astype('int'),
                y_test=y_test.astype('int'))

In [28]:
%time

model_LR_df_split = train_dev_test_split(X,y)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 14.3 µs


In [29]:
model_LR_df_split.keys()

dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])

In [30]:
model_LR_df_split['y_train'].unique()

array([ 0,  1, 13, 10,  3, 12,  5,  7,  8,  4,  2,  9,  6, 11])

In [ ]:
%%time

model_LR = Pipeline([
    ('tfidf', TfidfVectorizer(preprocessor=" ".join, ngram_range=(1,2))),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='sag', class_weight="balanced", multi_class='auto')))
    ])


model_LR.fit(model_LR_df_split['X_train'], model_LR_df_split['y_train'])

joblib.dump(model_LR, "model_lr.joblib")

In [83]:
!/Users/giuliagalli/google-cloud-sdk/bin/gsutil cp model_lr.joblib gs://reddit_models/

Copying file://model_lr.joblib [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][341.8 MiB/341.8 MiB]    7.8 MiB/s                                   
Operation completed over 1 objects/341.8 MiB.                                    


In [43]:
prediction = model_lr.predict(model_LR_df_split['X_test'])

loggg("prediction done")

[INFO] 2019-11-27 10:51:41.972883: prediction done


In [45]:
loggg("reporting Logistic Regression results")

print(classification_report(model_LR_df_split['y_test'], prediction))

[INFO] 2019-11-27 10:52:12.195361: reporting Logistic Regression results
              precision    recall  f1-score   support

           0       0.74      0.81      0.77    232289
           1       0.38      0.29      0.33    232243
           2       0.54      0.55      0.55    232080
           3       0.50      0.59      0.55    232145
           4       0.53      0.53      0.53    232007
           5       0.24      0.31      0.27    231810
           6       0.53      0.53      0.53    232122
           7       0.58      0.58      0.58    232173
           8       0.70      0.71      0.70    232026
           9       0.48      0.51      0.50    232104
          10       0.44      0.52      0.47    232721
          11       0.47      0.48      0.47    232131
          12       0.28      0.20      0.23    231997
          13       0.38      0.27      0.31    232157

   micro avg       0.49      0.49      0.49   3250005
   macro avg       0.49      0.49      0.49   3250005
weighte

# Testing the model

In [46]:
# gaming => correct

sentence1 = "What games would you like to discuss this week? [Game Thread Voting]"

In [47]:
tokenizer = TweetTokenizer()

text1 = tokenizer.tokenize(sentence1)

result1 = model_lr.predict([text1])

print(result1)

In [51]:
model_LR_df[model_LR_df['subreddit_id'] == int(result1)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
10059610,477973.0,"[sad, gotten, point, fan, better, game, aaa, s...",gaming,6
10059611,477974.0,"[pencil, refer]",gaming,6
10059612,477975.0,"[nake, gun, 33, 13, mayb]",gaming,6
10059613,477976.0,"[knight, old, republ, game]",gaming,6
10059614,477977.0,"[seen, redead]",gaming,6


In [52]:
# aww => wrong

sentence2 = "God bless you for introducing me to this sub"

In [76]:
tokenizer = TweetTokenizer()

text2 = tokenizer.tokenize(sentence2)

result2 = model_lr.predict([text2])

print(result2)

[2]


In [60]:
model_LR_df[model_LR_df['subreddit_id'] == int(result2)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
7737574,88650.0,"[argu, deep, human, except, thing, defianc, idea]",atheism,2
7737575,88651.0,"[differ, harri, potter, bibl, coupl, billion, ...",atheism,2
7737576,88652.0,"[explain, argument]",atheism,2
7737577,88653.0,"[driven, selfish]",atheism,2
7737578,88654.0,"[hear, god, heal, , sarcasm]",atheism,2


In [77]:
# todayilearned => correct

sentence3 = "TIL that conservationists created a match.com profile for the world’s loneliest frog named Romeo, in order to raise funds for expeditions to find more of his species. Their work paid off, and Romeo and Juliet are together at last."

In [78]:
tokenizer = TweetTokenizer()

text3 = tokenizer.tokenize(sentence3)

result3 = model_lr.predict([text3])

print(result3)

[12]


In [79]:
model_LR_df[model_LR_df['subreddit_id'] == int(result3)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
2321146,13706.0,"[ve, known, kill, box, corn, pop, meself]",todayilearned,12
2321147,13707.0,"[gtit, like, random, battleground, dayboth, be...",todayilearned,12
2321148,13708.0,"[ultim, beet, beetus]",todayilearned,12
2321149,13709.0,"[salt, fat, scum, rise, salt, pork, salt, beef...",todayilearned,12
2321150,13710.0,"[guy, right, need, correct, answer, know, nich...",todayilearned,12


In [80]:
# IAMA => wrong

sentence4 ="I do medical billing for a major hospital and I believe for-profit health care is evil. AMAA"

In [81]:
tokenizer = TweetTokenizer()

text4 = tokenizer.tokenize(sentence4)

result4 = model_lr.predict([text4])

print(result4)

[12]


In [82]:
model_LR_df[model_LR_df['subreddit_id'] == int(result4)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
2321146,13706.0,"[ve, known, kill, box, corn, pop, meself]",todayilearned,12
2321147,13707.0,"[gtit, like, random, battleground, dayboth, be...",todayilearned,12
2321148,13708.0,"[ultim, beet, beetus]",todayilearned,12
2321149,13709.0,"[salt, fat, scum, rise, salt, pork, salt, beef...",todayilearned,12
2321150,13710.0,"[guy, right, need, correct, answer, know, nich...",todayilearned,12
